## Decision Tree Exercises 

Using the titanic data, in your classification-exercises repository, create a notebook, model.ipynb where you will do the following:

In [156]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from pydataset import data

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import classification_report, confusion_matrix 


import matplotlib.pyplot as plt
import seaborn as sns

import acquire
import prepare

# Plan 
 - Let's Examine the Titanic DataSet
 - Can we accurately predict the survival of passengers on the Titanic based on categorical data, such as age, gender, passenger class, or fare.  

# Acquire
 - Acquire the data we have cleaned and prepped using our previous funtions.

In [157]:
df = acquire.get_titanic_data()

df.head()

Reading from csv file...


,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


# Prepare

In [158]:
df = prepare.prep_titanic(df)

In [159]:
df.head()

,survived,pclass,sex,sibsp,parch,fare,embark_town,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
0,0,3,male,1,0,7.2500,Southampton,0,1,0,1
1,1,1,female,1,0,71.2833,Cherbourg,0,0,0,0
2,1,3,female,0,0,7.9250,Southampton,1,0,0,1
3,1,1,female,1,0,53.1000,Southampton,0,0,0,1
4,0,3,male,0,0,8.0500,Southampton,1,1,0,1


In [160]:
df = df.drop(columns=["sex", "embark_town"])
df.head()

,survived,pclass,sibsp,parch,fare,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
0,0,3,1,0,7.2500,0,1,0,1
1,1,1,1,0,71.2833,0,0,0,0
2,1,3,0,0,7.9250,1,0,0,1
3,1,1,1,0,53.1000,0,0,0,1
4,0,3,0,0,8.0500,1,1,0,1


Prepare - Split Data

In [161]:
from sklearn.model_selection import train_test_split

In [162]:
def train_validate_test_split(df, target, seed=123):
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed, 
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test

In [163]:
train.shape

(498, 9)

In [164]:
validate.shape

(214, 9)

In [165]:
test.shape

(179, 9)

In [166]:
train, validate, test = train_validate_test_split(df, target='survived', seed=123)

X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

In [167]:
X_train.head()

,pclass,sibsp,parch,fare,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
583,1,0,0,40.1250,1,1,0,0
165,3,0,2,20.5250,0,1,0,1
50,3,4,1,39.6875,0,1,0,1
259,2,0,1,26.0000,0,0,0,1
306,1,0,0,110.8833,1,0,0,0


In [82]:
X_train.shape

(498, 8)

In [83]:
X_validate.shape

(214, 8)

In [119]:
X_test.shape

(179, 8)

In [120]:
y_train.value_counts()

0    307
1    191
Name: survived, dtype: int64

# Modeling

### Baseline

1.  a. What is your baseline prediction? 

    b. What is your baseline accuracy? 
    - remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [85]:
y_train[0:10]

583    0
165    1
50     0
259    1
306    1
308    0
314    0
883    0
459    0
180    0
Name: survived, dtype: int64

In [86]:
#obtain our mode (most occuring outcome)
train.survived.value_counts()

# baseline assumption = Did NOT survive

0    307
1    191
Name: survived, dtype: int64

In [87]:
# Obtain the mode for the target
baseline = y_train.mode()

# produce boolean array with True assigned to match the baseline prediction and real data. 
matches_baseline_prediction = (y_train == 0)

baseline_accuracy = matches_baseline_prediction.mean()

print(f'Baseline Accuracy: {round(baseline_accuracy, 2)}')

Baseline Accuracy: 0.62


### Fit - Transform
2. Fit the decision tree classifier to your training sample and transform 
- (i.e. make predictions on the training sample)

In [168]:
# Make the model
clf1 = DecisionTreeClassifier(max_depth=1, random_state=123)

#Fit the model (on train and only train)
clf1 = clf1.fit(X_train, y_train)

# Use the model
# We'll evaluate the model's performance on train first

y_predictions = clf1.predict(X_train)


In [169]:
print(export_text(tree, feature_names=X_train.columns.tolist()))

|--- sex_male <= 0.50
|   |--- pclass <= 2.50
|   |   |--- fare <= 28.86
|   |   |   |--- fare <= 28.23
|   |   |   |   |--- parch <= 0.50
|   |   |   |   |   |--- alone <= 0.50
|   |   |   |   |   |   |--- fare <= 22.50
|   |   |   |   |   |   |   |--- embark_town_Southampton <= 0.50
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- embark_town_Southampton >  0.50
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- fare >  22.50
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- alone >  0.50
|   |   |   |   |   |   |--- fare <= 12.82
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- fare >  12.82
|   |   |   |   |   |   |   |--- fare <= 13.25
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- fare >  13.25
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- parch >  0.50
|   |   |   |   |   |--- class: 1
|   |   |   |--- fare >  28.23
|   |   |   |   |--

### Evaluate Performance

3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [170]:
tree_score = clf1.score(X_train, y_train)
print(f'Accuracy of Decision Tree Classifier on trianing set: {tree_score:.2f}')

Accuracy of Decision Tree Classifier on trianing set: 0.80


In [132]:
pd.DataFrame(confusion_matrix(y_train, y_predictions))

,0,1
0,265,42
1,58,133


In [133]:
# Create a string object
classification_report(y_train, y_predictions)

'              precision    recall  f1-score   support\n\n           0       0.82      0.86      0.84       307\n           1       0.76      0.70      0.73       191\n\n    accuracy                           0.80       498\n   macro avg       0.79      0.78      0.78       498\nweighted avg       0.80      0.80      0.80       498\n'

In [134]:
# Creates a dataframe based off a dictionary

classification = classification_report(y_train, y_predictions, output_dict=True)
pd.DataFrame(classification).transpose()

,precision,recall,f1-score,support
0,0.820433,0.863192,0.841270,307.000000
1,0.760000,0.696335,0.726776,191.000000
accuracy,0.799197,0.799197,0.799197,0.799197
macro avg,0.790217,0.779764,0.784023,498.000000
weighted avg,0.797255,0.799197,0.797358,498.000000


### Additional - Calculate Metrics by Hand

4. Compute: 
    - Accuracy, 
    - true positive rate, 
    - false positive rate, 
    - true negative rate, 
    - false negative rate, 
    - precision, 
    - recall, 
    - f1-score, and 
    - support.

In [171]:
# Positives - Did NOT survive

TP = 276 
FP = 58
FN = 42
TN = 133
ALL = TP + FP + FN + TN

accuracy = (TP + TN) / ALL
print(f"Accuracy: {accuracy:.2f}")
      
true_positive_rate = TP / (TP + FN)
print(f"True Positive Rate: {true_positive_rate:.2f}")
      
false_positive_rate = FP /(FP + TN)
print(f"False Positive Rate: {false_positive_rate:.2f}")
      
true_negative_rate = TN / (TN + FP)
print(f"True Negative Rate: {true_negative_rate:.2f}")
      
false_negative_rate = FN / (FN + TP)
print(f"False Negative Rate: {false_negative_rate:.2f}")

precision = TP / (TP + FP)
print(f"Precision: {precision:.2f}")
            
recall = TP / ( TP + FN)
print(f"Recall: {recall:.2f}")

f1_score = 2 *(precision*recall) / (precision+recall)
print(f"F1: {f1_score:.2f}")
       
support_pos = TP + FN
print(f"F1: {support_pos:.2f}")
      
support_neg = FP + TN
print(f"F1: {support_neg:.2f}")


Accuracy: 0.80
True Positive Rate: 0.87
False Positive Rate: 0.30
True Negative Rate: 0.70
False Negative Rate: 0.13
Precision: 0.83
Recall: 0.87
F1: 0.85
F1: 318.00
F1: 191.00


In [172]:
classification = classification_report(y_train, y_predictions, output_dict=True)
pd.DataFrame(classification).transpose()

,precision,recall,f1-score,support
0,0.820433,0.863192,0.841270,307.000000
1,0.760000,0.696335,0.726776,191.000000
accuracy,0.799197,0.799197,0.799197,0.799197
macro avg,0.790217,0.779764,0.784023,498.000000
weighted avg,0.797255,0.799197,0.797358,498.000000


5. Run through steps 2-4 using a different max_depth value.

In [145]:
for i in range(2,10):

    tree = DecisionTreeClassifier(max_depth=i, random_state=123)

    tree = tree.fit(X_train, y_train)

    y_predictions = tree.predict(X_train)
    
    classification = classification_report(y_train, y_predictions, output_dict=True)

    print(f'Tree with a max depth of {i}')
    print(pd.DataFrame(classification).transpose())
    print("___________________")


Tree with a max depth of 2
              precision    recall  f1-score     support
0              0.820433  0.863192  0.841270  307.000000
1              0.760000  0.696335  0.726776  191.000000
accuracy       0.799197  0.799197  0.799197    0.799197
macro avg      0.790217  0.779764  0.784023  498.000000
weighted avg   0.797255  0.799197  0.797358  498.000000
___________________
Tree with a max depth of 3
              precision    recall  f1-score     support
0              0.828829  0.899023  0.862500  307.000000
1              0.812121  0.701571  0.752809  191.000000
accuracy       0.823293  0.823293  0.823293    0.823293
macro avg      0.820475  0.800297  0.807654  498.000000
weighted avg   0.822421  0.823293  0.820430  498.000000
___________________
Tree with a max depth of 4
              precision    recall  f1-score     support
0              0.829341  0.902280  0.864275  307.000000
1              0.817073  0.701571  0.754930  191.000000
accuracy       0.825301  0.825301  0.82

6. Which model performs better on your in-sample data?

The model with a max depth of 9 performs best with an accuracy of 91%. As the depth increases, the accuracy increases, which aslo demonstrates how models can be overfit to the data. 

### Validation
7. Which model performs best on your out-of-sample data, the validate set?

In [179]:
metrics = []

for i in range (2, 20):
        # Make the model
        tree = DecisionTreeClassifier(max_depth=i, random_state=123)
        
        #Fit the model on TRAIN only)
        tree = tree.fit(X_train, y_train)
        
        #Use the model - on train first, then on validate
        in_sample_accuracy = tree.score(X_train, y_train)
        
        out_of_sample_accuracy = tree.score(X_validate, y_validate)
        
        output = {
            "max_depth": i,
            "train_accuracy": in_sample_accuracy,
            "validate_accuracy": out_of_sample_accuracy
        }
        
        metrics.append(output)
        
df = pd.DataFrame(metrics)
df['difference'] = df.train_accuracy - df.validate_accuracy

df
    

,max_depth,train_accuracy,validate_accuracy,difference
0,2,0.799197,0.761682,0.037515
1,3,0.823293,0.785047,0.038246
2,4,0.825301,0.785047,0.040254
3,5,0.837349,0.757009,0.080340
4,6,0.859438,0.766355,0.093083
5,7,0.863454,0.761682,0.101772
6,8,0.897590,0.757009,0.140581
7,9,0.909639,0.761682,0.147956
8,10,0.923695,0.766355,0.157340
9,11,0.931727,0.761682,0.170045


In [181]:
df[df.difference <= 0.10].sort_values(by=['validate_accuracy', 'difference'])

,max_depth,train_accuracy,validate_accuracy,difference
3,5,0.837349,0.757009,0.080340
0,2,0.799197,0.761682,0.037515
4,6,0.859438,0.766355,0.093083
1,3,0.823293,0.785047,0.038246
2,4,0.825301,0.785047,0.040254


## Random Forests

1. Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

2. Evaluate your results using the model score, confusion matrix, and classification report.

3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

4. Run through steps increasing your min_samples_leaf and decreasing your max_depth.

5. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

After making a few models, which one has the best performance (or closest metrics) on both train and validate?

### K-Nearest Neighbors

1. Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

2. Evaluate your results using the model score, confusion matrix, and classification report.

3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

4. Run through steps 2-4 setting k to 10

5. Run through setps 2-4 setting k to 20

6. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

7. Which model performs best on our out-of-sample data from validate?